In [ ]:
# https://openrouter.ai/settings/keys

In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install pdfplumber

In [ ]:
!pip install faiss-cpu

In [ ]:
!git clone https://github.com/deepanrajm/deep_learning.git

Cloning into 'deep_learning'...
remote: Enumerating objects: 2972, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 2972 (delta 83), reused 247 (delta 39), pack-reused 2667 (from 2)
Receiving objects: 100% (2972/2972), 308.69 MiB | 25.95 MiB/s, done.
Resolving deltas: 100% (207/207), done.
Updating files: 100% (2611/2611), done.


In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Step 1: Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"  # Replace with your actual OpenRouter API key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Step 2: Initialize the LLM
llm = ChatOpenAI(
    model="mistralai/mistral-small-3.2-24b-instruct:free",  # or try "meta-llama/llama-3-8b-instruct", etc.
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
    verbose=True
)

# Step 3: Interact with the model (normal chat mode)
while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    response = llm([HumanMessage(content=query)])
    print("\nAssistant:", response.content)


/tmp/ipython-input-2-1351599517.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/tmp/ipython-input-2-1351599517.py:26: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=query)])



Assistant: **Hare Krishna!**

The mantra **"Hare Krishna"** is a sacred chant from the *Bhagavad Gita* and *Srimad Bhagavatam*, central to the devotional traditions of Hinduism, particularly in *Gaudiya Vaishnavism* (the lineage of Chaitanya Mahaprabhu).

The full **maha-mantra** is:
**"Hare Krishna Hare Krishna, Krishna Krishna Hare Hare, Hare Rama Hare Rama, Rama Rama Hare Hare."**

### **Meaning & Significance:**
- **Hare** – An address to the divine energy (Shakti) of Krishna.
- **Krishna** – The Supreme Personality of Godhead, the blissful creator.
- **Rama** – Another name for the Supreme Lord (as in *Rama* of Ayodhya or *Balarama* in Vaishnavism).

This mantra is believed to purify the heart, connect devotees to God, and bring spiritual liberation (*moksha*) through devotion (*bhakti*).

### **How It’s Used:**
- Chanted as a meditation or prayer.
- Popularized by the **International Society for Krishna Consciousness (ISKCON)**.
- Sung in temples, festivals, and personal worship

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Load and process PDF
loader = PDFPlumberLoader("/content/deep_learning/RAG/Basic_Home_Remedies.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",   #"mistralai/mistral-7b-instruct"
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# Prompt
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is a natural remedy for cough mentioned in the document?")
print("Answer:", result["result"])


In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from copy import deepcopy

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Load and process PDF
loader = PDFPlumberLoader(r"/content/deep_learning/RAG/Basic_Home_Remedies.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
semantic_chunks = text_splitter.split_documents(docs)

# manually apply overlap across those semantic chunks
def apply_overlap_to_chunks(chunks, overlap_characters=200):
    overlapped_chunks = []

    for i in range(len(chunks)):
        curr = deepcopy(chunks[i])
        if i > 0:
            prev = chunks[i - 1].page_content
            # Get the last N characters of the previous chunk
            overlap = prev[-overlap_characters:]
            # Prepend to current chunk
            curr.page_content = overlap + curr.page_content
        overlapped_chunks.append(curr)

    return overlapped_chunks

# Apply overlap (adjust 200 to suit your context)
documents = apply_overlap_to_chunks(semantic_chunks, overlap_characters=200)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",   #"mistralai/mistral-7b-instruct"
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# Prompt
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is a natural remedy for cough mentioned in the document?")
print("Answer:", result["result"])
